# Usage guide

This notebook shows how to assemble SQL queries using FunSQL. It introduce the different constructs available and how to compose them. 

**NOTE**: This document plagiarizes heavily from the corresponding guide for the [original library](https://mechanicalrabbit.github.io/FunSQL.jl/stable/guide) in Julia.

For illustration, we use the dataset used in the web project, [SQL Murder Mystery](https://mystery.knightlab.com/), which is a really fun way to practice SQL.  The premise is that a crime has taken place, and you go through the data to nail down the suspects.  All records are available as tables in a _sqlite_ database file, which is included in the `docs` directory. 

![Database schema](assets/mystery_schema.png)

Most tables are personal records, visits to a fitness centre, or interactions with the police. To interact with the data, we use the python stadard library for sqlite. 

In [1]:
import sqlite3

conn = sqlite3.connect("assets/mystery_database.db")


def run_query(q_str):
    curr = conn.cursor()
    curr.execute(q_str)
    return curr.fetchall()


# list of all tables
_curr = conn.cursor()
_curr.execute("SELECT name FROM sqlite_master where type = 'table'")
_curr.fetchall()

[('crime_scene_report',),
 ('drivers_license',),
 ('person',),
 ('facebook_event_checkin',),
 ('interview',),
 ('get_fit_now_member',),
 ('get_fit_now_check_in',),
 ('income',),
 ('solution',)]

## Using FunSQL

To compile queries, FunSQL needs to know schemas of the corresponding tables and the sql dialect to generate. This information is put into a `SQLCatalog` object, which can be reused across queries. 

In [2]:
from funsql import *
from funsql.tools import dialect_sqlite, make_sql_tables, reflect_sqlite

In [3]:
# `reflect_sqlite` is an SQL query to fetch all table schemas in the database
_curr = conn.cursor()
_curr.execute(reflect_sqlite())
list_tables = _curr.fetchall()

dialect = dialect_sqlite()
catalog = SQLCatalog(dialect, {tab.name: tab for tab in make_sql_tables(list_tables)})

We created an SQLCatalog object with all database tables mapped to their original names.  To get the query results formatted neatly, we use the pandas library since Jupyter knows how to render dataframes. 

In [4]:
import pandas as pd


def render_query(query):
    """returns an SQL string which can be executed"""
    return render(query, catalog=catalog)


def run_fun_query(query):
    """take a FunSQL node object and return results"""
    q_str = render_query(query).query
    return pd.read_sql_query(q_str, conn)


def run_query(q_str):
    """take a SQL query string and return results"""
    return pd.read_sql_query(q_str, conn)

We are good to go. For a first query, lets look at all personal records. 

In [5]:
q = From(S.person)
run_fun_query(q)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


## Why FunSQL?

Consider a question from the given dataset. For example, 

_Find all people and the cars they drive, who live somewhere on `Franklin Ave` and are between 30 and 40 years old._

It can be answered with the following SQL query. 

```sql
SELECT name, car_make FROM person 
JOIN drivers_license AS license
ON license_id = license.id
WHERE (license.age BETWEEN 30 AND 40)
AND address_street_name LIKE '%Franklin Ave%'
```

The execute this query from python code, we can embed it as a string literal. 

In [6]:
sql = """
SELECT name, car_make FROM person 
JOIN drivers_license AS license
ON license_id = license.id
WHERE (license.age BETWEEN 30 AND 40)
AND address_street_name LIKE '%Franklin Ave%'
"""

run_query(sql)

,name,car_make
0,Annabel Miller,Toyota
1,Shelby Dezeeuw,Toyota
2,Whitney Herkenratt,Maserati
3,Maria Walsh,Chevrolet
4,Raul Eads,Chevrolet


With FunSQL, instead of embedding the SQL query directly into code, we construct a query object. 

In [7]:
q = (
    From(S.person)
    >> Join(
        From(S.drivers_license) >> Where(Fun.between(Get.age, 30, 40)) >> As(S.l),
        on=Fun("=", Get.license_id, Get.l.id),
    )
    >> Where(Fun.LIKE(Get.address_street_name, "%Franklin Ave%"))
    >> Select(Get.name, Get.l.car_make)
)

The value of q is a composite object of type `SQLNode`. "Composite" means that q is assembled from components (also of type `SQLNode`), which themselves are either atomic or assembled from smaller components. Different kinds of components are created by `SQLNode` constructors such as `From`, `Where`, `Fun`, `Get`, etc.

We can then serialize the query object as a SQL statement and execute it. 

In [8]:
run_fun_query(q)

,name,car_make
0,Annabel Miller,Toyota
1,Shelby Dezeeuw,Toyota
2,Whitney Herkenratt,Maserati
3,Maria Walsh,Chevrolet
4,Raul Eads,Chevrolet


Why write a query indirectly, instead of executing the original SQL string? Consider that, in an application, queries are generally parameterized. The previous question will look like, 

_Find all people and the cars they drive, who live somewhere on {address} and are between {start_age} and {end_age} years old._


This is still solvable by introducing placeholder parameters in the query string, and providing them at runtime. 

In [9]:
sql = """
SELECT name, car_make FROM person 
JOIN drivers_license AS license
ON license_id = license.id
WHERE (license.age BETWEEN ? AND ?)
AND address_street_name LIKE ?
"""
pd.read_sql_query(sql, conn, params=[30, 40, "Franklin Ave"])

,name,car_make
0,Annabel Miller,Toyota
1,Shelby Dezeeuw,Toyota
2,Whitney Herkenratt,Maserati
3,Maria Walsh,Chevrolet
4,Raul Eads,Chevrolet


However, say if we don't want to use the age filter now, or maybe use a different table to get people ages since the license information migth be outdated. Then, the SQL query cannot be prepared in advance and must be assembled on the fly using string concatenation, which is error prone. Building the query as a python data structure is verbose but robust. 

The FunSQL query can be constructed as below. 

In [10]:
def find_people(start_age, end_age, address):
    q = From(S.person) >> Join(
        From(S.drivers_license)
        >> Where(age_range(Get.age, start_age, end_age))
        >> As("l"),
        on=Fun("=", Get.license_id, Get.l.id),
    )
    if address is not None:
        q = q >> Where(Fun.LIKE(Get.address_street_name, address))
    return q >> Select(Get.name, Get.l.car_make)


def age_range(age_col, start_age, end_age):
    if start_age is None and end_age is None:
        return True
    elif start_age is None:
        return Fun("<=", age_col, end_age)
    elif end_age is None:
        return Fun(">=", age_col, start_age)
    else:
        return Fun.BETWEEN(age_col, start_age, end_age)

In [11]:
run_fun_query(find_people(30, 40, "%Franklin Ave%"))

,name,car_make
0,Annabel Miller,Toyota
1,Shelby Dezeeuw,Toyota
2,Whitney Herkenratt,Maserati
3,Maria Walsh,Chevrolet
4,Raul Eads,Chevrolet


The function `find_people` is effectively a new `SQLNode` constructor, which can be used directly, or as a component of a larger query. Queries are now easier to tweak. 

_List all people living on Franklin Ave_ (no age filter)

In [12]:
run_fun_query(find_people(None, None, "Franklin Ave"))

,name,car_make
0,Wilmer Wolever,Nissan
1,Annabel Miller,Toyota
2,Johnnie Schee,Honda
3,Carleen Etoll,Mitsubishi
4,Zachary Ybarbo,Ford
5,Gema Nantz,BMW
6,Clarita Rickels,Chrysler
7,Shelby Dezeeuw,Toyota
8,Amado Mattan,Acura
9,Cordell Lindamood,Toyota


_List people less than 25 years old, living on Northwestern Dr_

In [13]:
run_fun_query(find_people(None, 25, "Northwestern Dr"))

,name,car_make
0,Muoi Cary,Mercedes-Benz
1,Norman Apolito,Mazda
2,Abe Roeker,Dodge
3,Courtney Bordeaux,Mercury


## Tabular operations

Lets look at the query from the previous section again. 

_Find all people and the cars they drive, who live somewhere on `Franklin Ave` and are between 30 and 40 years old._

In [14]:
q = (
    From(S.person)
    >> Join(
        From(S.drivers_license) >> Where(Fun.between(Get.age, 30, 40)) >> As(S.l),
        on=Fun("=", Get.license_id, Get.l.id),
    )
    >> Where(Fun.LIKE(Get.address_street_name, "%Franklin Ave%"))
    >> Select(Get.name, Get.l.car_make)
)

At the outer level, this query is constructed from tabular operations From, Where, Join, and Select arranged in a pipeline by the pipe (>>) operator. In SQL, a tabular operation takes a certain number of input datasets and produces an output dataset. 

It is helpful to visualize a tabular operation as a node with a certain number of input arrows and one output arrow.

<img src="assets/query-nodes.svg" alt="query nodes" width=800 />

Then the whole query can be visualized as a pipeline diagram. Each arrow in this diagram represents a dataset, and each node represents an elementary data processing operation.

<img src="assets/funsql-example-query.svg" alt="query nodes" />

The following tabular operations are available in FunSQL.

* **Append** - concatenate datasets
* **As** - wrap all columns in a nested record
* **Define** - add an output column
* **From** - produce the content of a database table
* **Group** - partition the dataset into disjoint groups
* **Iterate** - iterate a query
* **Join** - correlate two datasets
* **Limit** - truncate the dataset
* **Order** - sort the dataset
* **Partition** - relate dataset rows to each other
* **Select** - specify output columns
* **Where** - filter the dataset by the given condition
* **With** - assign a name to a temporary dataset


## From, Select and Define

The `From` node outputs the content of a database table. The constructor requires an `SQLTable` object that lists out the columns present in the table. It can either be an explicit SQLTable, or the name of an entry in the `SQLCatalog` (which lists all tables available). 

FunSQL queries don't need to have an explicit Select, and all columns available at the final node are output. For example, just rendering a `From` node lists all columns in the table. 

In [15]:
q = From(S.person)
render_query(q)

query: 
SELECT
  "person_1"."id", 
  "person_1"."name", 
  "person_1"."license_id", 
  "person_1"."address_number", 
  "person_1"."address_street_name", 
  "person_1"."ssn"
FROM "person" AS "person_1"

When the query isn't made against a table, the `From` node can be skipped. For ex, a query to show the current timestamp. 

In [16]:
q = Select(Fun.current_timestamp())
render_query(q)

query: 
SELECT CURRENT_TIMESTAMP AS "current_timestamp"

The `Select` node is used to specify the columns to output from a query. The name of the columns are either derived from the expression or set explicitly with `As`. 

In [17]:
q = From(S.person) >> Select(Get.name, Get.address_street_name >> As(S.street))
render_query(q)

query: 
SELECT
  "person_1"."name", 
  "person_1"."address_street_name" AS "street"
FROM "person" AS "person_1"

New columns can be added to a query using the `Define` node. 

_For all fitness centre members, find out how long have they been active_.

In [18]:
q = (
    From(S.person)
    >> Join(
        From(S.get_fit_now_member) >> As(S.gym), Fun("=", Get.id, Get.gym.person_id)
    )
    >> Define(
        Fun("/", Fun("-", 20220000, Get.gym.membership_start_date), 10_000)
        >> As("member_for_years")
    )
    >> Select(Get.name, Get.member_for_years)
)
render_query(q)

query: 
SELECT
  "person_1"."name", 
  ((20220000 - "get_fit_now_member_1"."membership_start_date") / 10000) AS "member_for_years"
FROM "person" AS "person_1"
INNER JOIN "get_fit_now_member" AS "get_fit_now_member_1" ON ("person_1"."id" = "get_fit_now_member_1"."person_id")

The new column replaces an existing column of the same name. 

_Get names from all person records with salutations removed_.

In [19]:
q = From(S.person) >> Define(Fun.LTRIM(Get.name, "Dr. ") >> As(S.name))

render_query(q)

query: 
SELECT
  "person_1"."id", 
  "person_1"."license_id", 
  "person_1"."address_number", 
  "person_1"."address_street_name", 
  "person_1"."ssn", 
  LTRIM("person_1"."name", 'Dr. ') AS "name"
FROM "person" AS "person_1"

## Join

The `Join` node is used to pull together related rows from two input datasets.  In the following example, Join associates each person record with their driver's license using the key column `license_id` that uniquely identifies a license record.

_Show all people along with their driver's license information_.

In [20]:
q = (
    From(S.person)
    >> Join(
        From(S.drivers_license) >> As(S.l),
        on=Fun("=", Get.license_id, Get.l.id),
        left=True,
    )
    >> Select(Get.name, Get.license_id, Get.l.car_make)
)
render_query(q)

query: 
SELECT
  "person_1"."name", 
  "person_1"."license_id", 
  "drivers_license_1"."car_make"
FROM "person" AS "person_1"
LEFT JOIN "drivers_license" AS "drivers_license_1" ON ("person_1"."license_id" = "drivers_license_1"."id")

The type of the join can be specified by passing different values for the boolean arguments `left` and `right` to the Join node. 

* INNER: `left=False, right=False`
* LEFT: `left=True, right=False`
* RIGHT: `left=False, right=True`
* OUTER: `left=True, right=True`

A Join needs two input datasets, so it must be attached to two input pipelines. The first pipeline is attached using the `>>` operator and the second one is provided as an argument to the Join constructor. 

Alternatively, both input pipelines can be specified as keyword arguments, since the pipeline operator (`>>`) really works by setting the `over` attribute on the right hand side node. 

In [21]:
q = Join(
    over=From(S.person),
    joinee=From(S.drivers_license) >> As(S.l),
    on=Fun("=", Get.license_id, Get.l.id),
    left=True,
)

The output of Join combines columns of both input datasets, which will cause ambiguity if both datasets have a column with the same name. For example, both tables `person` and `license`, have a column called `id`. 

To disambiguate them, we can place all columns of one of the datasets into a nested record, using the `As` node. 

In [22]:
q = (
    Join(
        over=From(S.person),
        joinee=From(S.drivers_license) >> As(S.l),
        on=Fun("=", Get.license_id, Get.l.id),
        left=True,
    )
    >> Select(Get.name, Get.license_id, Get.l.id, Get.l.car_make)
)

In the output of the join, columns from `drivers_license` are nested inside the field named "l" and can only be accessed as, `Get.l.*`. 

## Scalar operations

Many tabular operations including Join, Select and Where are parameterized with scalar operations. A scalar operation acts on an individual row of a dataset and produces a scalar value. Scalar operations are assembled from literal values, column references, and applications of SQL functions and operators. Below is a list of scalar operations available in FunSQL.

* Agg - apply an aggregate function
* As - assign a column alias
* Bind - create a correlated subquery
* Fun - apply a scalar function or a scalar operator
* Get - produce the value of a column
* Lit - produce a constant value
* Sort - indicate the sort order
* Var - produce the value of a query parameter

## Lit: SQL Literals

The `Lit` node creates a literal value - numbers, timestamps, strings. 

In [23]:
Select(Lit(20))

Select(Lit(20))

In [24]:
Select(Lit("hello world"))

Select(Lit("hello world"))

When python literals are passed as arguments to node constructors, FunSQL casts them to a `Lit` node automatically. 

In [25]:
Select("hello world")

Select(Lit("hello world"))

In [26]:
Fun("=", 100, 200)

Fun."="(Lit(100), Lit(200))

The SQL value NULL is represented by the Pyhton `None`. 

In [27]:
render(Select(None))

query: 
SELECT NULL AS "_"

## Get: Column References

The Get node creates a column reference, and can be created by passing the column name to the `Get` constructor. 

In [28]:
Get(S.license_id)

Get.license_id

The `S.*` syntax is used to construct a "symbol", so FunSQL can distinguish literal string values from strings that refer to tables or columns. 

Most places where it is clear the string is a reference, FunSQL casts the input string to a symbol itself.

In [29]:
Get("license_id")

Get.license_id

`Get` nodes can also be constructed using the _dot_ syntax. 

In [30]:
Get.license_id

Get.license_id

It works for nested references too. 

In [31]:
Get.alias.license_id

Get.alias.license_id

FunSQL resolves column references at the place of use against the input dataset. This is much like SQL where the column names reference the dataset/query referred by the corresponding `From` clause. 

When the compiler can't resolve a reference umabiguously, it raises an error. Like below, both tables `person` and `drivers_license` have a column called `id`, so the compiler can't deduce which one the join condition is referring to. 

In [32]:
# query = From(S.person) >> Join(
#     From(S.drivers_license), on=Fun("=", Get.id, Get.license_id)
# )
# render_query(query)

As pointed out with the `Join` node, one way to disambiguate columns is nesting column references for one of the tables in an alias. 

In [33]:
query = From(S.person) >> Join(
    From(S.drivers_license) >> As(S.alias), on=Fun("=", Get.alias.id, Get.license_id)
)
render_query(query)

query: 
SELECT
  "person_1"."address_street_name", 
  "person_1"."id", 
  "person_1"."address_number", 
  "person_1"."name", 
  "person_1"."ssn", 
  "person_1"."license_id"
FROM "person" AS "person_1"
INNER JOIN "drivers_license" AS "drivers_license_1" ON ("drivers_license_1"."id" = "person_1"."license_id")

Another option is to bind the column reference to the node that produces it. 

In [34]:
q1 = From(S.person)
q2 = From(S.drivers_license)
query = q1 >> Join(q2, on=Fun("=", q2 >> Get.id, Get.license_id))
render_query(query)

query: 
SELECT
  "person_1"."address_street_name", 
  "person_1"."address_number", 
  "person_1"."name", 
  "person_1"."ssn", 
  "person_1"."license_id", 
  "drivers_license_1"."hair_color", 
  "drivers_license_1"."car_model", 
  "drivers_license_1"."plate_number", 
  "drivers_license_1"."eye_color", 
  "drivers_license_1"."age", 
  "drivers_license_1"."gender", 
  "drivers_license_1"."car_make", 
  "drivers_license_1"."height"
FROM "person" AS "person_1"
INNER JOIN "drivers_license" AS "drivers_license_1" ON ("drivers_license_1"."id" = "person_1"."license_id")

## Fun: SQL Functions and Operators

SQL functions and operators are represented using the Fun node, which can be created by passing the operator first, and the arguments after. 

In [35]:
Fun(">=", Get.age, 40)

Fun.">="(Get.age, Lit(40))

Though if the operator is a string, `Fun` nodes allow the _dot_ syntax. 

In [36]:
Fun.between(Get.age, 30, 40)

Fun.between(Get.age, Lit(30), Lit(40))

In [37]:
Fun.LIKE(Get.address_street_name, "%Franklin Ave%")

Fun.LIKE(Get.address_street_name, Lit("%Franklin Ave%"))

FunSQL is oblivious to if the SQL function or operator is used correctly or even whether it is supported by the execution engine. It just prints out the function expression in the final SQL query, which might fail to execute.

In [38]:
query = From(S.person) >> Select(Fun.unavailable(Get.name))
render_query(query)

# run_fun_query(query)
# OperationalError: no such function: UNAVAILABLE

query: 
SELECT UNAVAILABLE("person_1"."name") AS "unavailable"
FROM "person" AS "person_1"

On the other hand, FunSQL will correctly serialize many SQL functions and operators that have irregular syntax including AND, OR, NOT, IN, EXISTS, CASE, and others.

_Categorize driving license holders by their age_.

In [39]:
query = (
    From(S.person)
    >> Join(From(S.drivers_license) >> As(S.l), on=Fun("=", Get.l.id, Get.license_id))
    >> Select(
        Get.name,
        Fun.CASE(Fun(">", Get.l.age, 21), "adult", "not yet") >> As(S.category),
    )
)
render_query(query)

query: 
SELECT
  "person_1"."name", 
  (CASE WHEN ("drivers_license_1"."age" > 21) THEN 'adult' ELSE 'not yet' END) AS "category"
FROM "person" AS "person_1"
INNER JOIN "drivers_license" AS "drivers_license_1" ON ("drivers_license_1"."id" = "person_1"."license_id")

## Group and Aggregate Functions

Group and aggregate functions are used for summarizing data to report totals, averages and so on. 

FunSQL uses the `Group` node, which partitions the input dataset into disjoint groups. Then, for each group, we can calculate summary values using aggregate functions.  Aggregate functions are created using the `Agg` node. 

_Count the number of people living on each street_.

In [40]:
query = (
    From(S.person)
    >> Group(Get.address_street_name)
    >> Select(Get.address_street_name, Agg.count())
)
render_query(query)

query: 
SELECT
  "person_1"."address_street_name", 
  count(*) AS "count"
FROM "person" AS "person_1"
GROUP BY "person_1"."address_street_name"

To indicate that aggregate functions must be applied to the dataset as a whole, we create a Group node without arguments. 

**NOTE**: This is different from regular SQL, where a GROUP BY clause is not needed when aggregating over the full dataset. However, FunSQL needs an empty group node to resolve aggregate expressions correctly. 

_Get the average age of a driving license holder._

In [41]:
query = From(S.drivers_license) >> Group() >> Select(Agg.avg(Get.age))
render_query(query)

query: 
SELECT avg("drivers_license_1"."age") AS "avg"
FROM "drivers_license" AS "drivers_license_1"

Like regular SQL, you can also group by multiple attributes by passing multiple column references to the `Group` node. The output of `Group` in that case contains all distinct values of the _grouping key_.

_Find the number of crime scene reports by type for each day._

In [42]:
query = (
    From(S.crime_scene_report)
    >> Group(Get.date, Get.type)
    >> Select(Get.date, Get.type, Agg.count())
)
render_query(query)

query: 
SELECT
  "crime_scene_report_1"."date", 
  "crime_scene_report_1"."type", 
  count(*) AS "count"
FROM "crime_scene_report" AS "crime_scene_report_1"
GROUP BY
  "crime_scene_report_1"."date", 
  "crime_scene_report_1"."type"

FunSQL has no lexical limitations on the use of aggregate functions. While in SQL, aggregate functions can only be used in the SELECT or HAVING clauses, there is no such restriction in FunSQL; they could be used in any context where an ordinary expression is permitted. 

The only requirement is that for each aggregate function, FunSQL should be able to resolve which `Group` node it refers to.  It is convenient to imagine the output of Group contains the grouped rows, which cannot be observed directly, but whose presence in the output allows us to apply aggregate functions.

For example, to filter grouped records, you can use a regular `Where` node, whereas SQL would require a HAVING clause.

_Fetch all crime reports with more than two incidents of the same type in a day._

In [43]:
query = (
    From(S.crime_scene_report)
    >> Group(Get.date, Get.type)
    >> Where(Fun(">", Agg.count(), 2))
)
render_query(query)

query: 
SELECT
  "crime_scene_report_1"."date", 
  "crime_scene_report_1"."type"
FROM "crime_scene_report" AS "crime_scene_report_1"
GROUP BY
  "crime_scene_report_1"."date", 
  "crime_scene_report_1"."type"
HAVING (count(*) > 2)

When the output of Group is _nested_ inside an alias using an `As` node, you need to traverse it with `Get` in order to use an aggregate function.

_When was the last time each `Get Fit Now` member visited the gym?_

In [44]:
query = (
    From(S.get_fit_now_member)
    >> Join(
        From(S.get_fit_now_check_in) >> Group(Get.membership_id) >> As(S.check_ins),
        on=Fun("=", Get.id, Get.check_ins.membership_id),
        left=True,
    )
    >> Select(Get.name, Get.check_ins >> Agg.max(Get.check_in_date))
)

render_query(query)

query: 
SELECT
  "get_fit_now_member_1"."name", 
  "check_ins_1"."max"
FROM "get_fit_now_member" AS "get_fit_now_member_1"
LEFT JOIN (
  SELECT
    "get_fit_now_check_in_1"."membership_id", 
    max("get_fit_now_check_in_1"."check_in_date") AS "max"
  FROM "get_fit_now_check_in" AS "get_fit_now_check_in_1"
  GROUP BY "get_fit_now_check_in_1"."membership_id"
) AS "check_ins_1" ON ("get_fit_now_member_1"."id" = "check_ins_1"."membership_id")

## Partition and Window Functions

We can relate each row to other rows in the same dataset using the `Partition` node and `window` functions. 

The `Partition` node works similar to the `Group` node by partitioning the input rows into disjoint groups. However, unlike `Group`, which collapses each row group into a single row, the Partition node preserves the original rows, but allows us to relate each row to adjacent rows in the same partition. In particular, we can apply regular aggregate functions, which calculate the summary value of a subset of rows related to the current row.

In the following example, the rows in `get_fit_now_check_in` table are partitioned by `check_in_date` and ordered by the `check_in_time` in increasing order. The frame clause specifies the subset of rows relative to the current row (the window frame) to be used by aggregate functions. In this example, the frame contains all rows prior to the current row.

_For each check in to the fitness centre, show the number of people who checked in before on the same day._

In [45]:
query = (
    From(S.get_fit_now_check_in)
    >> Partition(
        Get.check_in_date,
        order_by=[Get.check_in_time],
        frame=Frame(F.ROWS, F.pre(None), F.pre(1)),
    )
    >> Select(Get.membership_id, Get.check_in_date, Agg.count())
)
render_query(query)

query: 
SELECT
  "get_fit_now_check_in_1"."membership_id", 
  "get_fit_now_check_in_1"."check_in_date", 
  (count(*) OVER (PARTITION BY "get_fit_now_check_in_1"."check_in_date" ORDER BY "get_fit_now_check_in_1"."check_in_time" ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)) AS "count"
FROM "get_fit_now_check_in" AS "get_fit_now_check_in_1"

## Query Parameters

A SQL query may include a reference to a query parameter. The actual values for all parameters used in the query must then be provided at runtime. 

FunSQL can be used to construct a query with parameters. Similar to Get, parameter references are created using the Var node. 

_Find all driving license holders between 30 and 40 years of age._

In [46]:
query = (
    From(S.person)
    >> Join(From(S.drivers_license) >> As(S.l), on=Fun("=", Get.license_id, Get.l.id))
    >> Where(Fun.between(Get.l.age, Var.age_lower, Var.age_upper))
    >> Select(Get.name)
)

render_query(query)

query: 
SELECT "person_1"."name"
FROM "person" AS "person_1"
INNER JOIN "drivers_license" AS "drivers_license_1" ON ("person_1"."license_id" = "drivers_license_1"."id")
WHERE ("drivers_license_1"."age" BETWEEN ?1 AND ?2)

vars: [age_lower, age_upper]

FunSQL serializes the placeholder for parameter values as per the dialect being used. You can provide parameter values to the execution engine to evaluate the query. 

In [47]:
q_str = render_query(query).query  # get only the sql string
pd.read_sql_query(q_str, conn, params=[30, 40])

,name
0,Cory Rodriquz
1,Dayna Hamiter
2,Marvella Rummler
3,Everette Arlia
4,Bruna Sumlin
...,...
1576,Kurt Leota
1577,Hattie Hertel
1578,Colene Newland
1579,Kirk Samperi


## Correlated queries

An inner query is a SQL query that is included into the outer query as a part of a scalar expression. An inner query must either produce a single value or be used as an argument of a query operator, such as IN or EXISTS, which transforms the query output to a scalar value.

FunSQL lets you create inner queries like regular SQL. 

_Find the youngest driving license holders._

In [48]:
q1 = From(S.drivers_license) >> Group() >> Select(Agg.min(Get.age))

query = (
    From(S.drivers_license) >> Where(Fun("=", Get.age, q1)) >> Select(Get.id, Get.age)
)
render_query(query)

query: 
SELECT
  "drivers_license_1"."id", 
  "drivers_license_1"."age"
FROM "drivers_license" AS "drivers_license_1"
WHERE ("drivers_license_1"."age" = (
  SELECT min("drivers_license_2"."age") AS "min"
  FROM "drivers_license" AS "drivers_license_2"
))

The inner query may depend on the data from the outer query. Such inner queries are called **correlated**. 

In FunSQL, correlated queries are created using the `Bind` node. Specifically, in the body of a correlated query we use query parameters to refer to the external data. The Bind node, which wrap the correlated query, binds each parameter to an expression evaluated in the context of the outer query.

_Find fitness centre members who spent more than 100 minutes at the centre some time._

In [49]:
visits_over_duration = (
    From(S.get_fit_now_check_in)
    >> Where(Fun(">", Fun("-", Get.check_out_time, Get.check_in_time), Var.DURATION))
    >> Where(Fun("=", Get.membership_id, Var.MEMBER_ID))
)

query = (
    From(S.get_fit_now_member)
    >> Where(
        Fun.exists(
            visits_over_duration
            >> Bind(Get.id >> As(S.MEMBER_ID), Lit(100) >> As(S.DURATION))
        )
    )
    >> Select(Get.name)
)

render_query(query)

query: 
SELECT "get_fit_now_member_1"."name"
FROM "get_fit_now_member" AS "get_fit_now_member_1"
WHERE (EXISTS (
  SELECT NULL
  FROM "get_fit_now_check_in" AS "get_fit_now_check_in_1"
  WHERE ((("get_fit_now_check_in_1"."check_out_time" - "get_fit_now_check_in_1"."check_in_time") > 100) AND ("get_fit_now_check_in_1"."membership_id" = "get_fit_now_member_1"."id"))
))

## Order and Limit

To get output rows from a query in a specific order, FunSQL provides the `Order` node. 

_List all the Toyota owners by age._

In [50]:
query = (
    From(S.person)
    >> Join(From(S.drivers_license) >> As(S.l), on=Fun("=", Get.l.id, Get.license_id))
    >> Where(Fun.LIKE(Get.l.car_make, "%Toyota%"))
    >> Select(Get.name, Get.l.age >> As(S.age))
    >> Order(Get.age)
)

render_query(query)

query: 
SELECT
  "person_2"."name", 
  "person_2"."age"
FROM (
  SELECT
    "person_1"."name", 
    "drivers_license_1"."age"
  FROM "person" AS "person_1"
  INNER JOIN "drivers_license" AS "drivers_license_1" ON ("drivers_license_1"."id" = "person_1"."license_id")
  WHERE ("drivers_license_1"."car_make" LIKE '%Toyota%')
) AS "person_2"
ORDER BY "person_2"."age"

The Asc and the Desc modifiers specify whether to sort the rows in an ascending or in a descending order.

_List all the Toyota owners by age, in decreasing order._

In [51]:
query = (
    From(S.person)
    >> Join(From(S.drivers_license) >> As(S.l), on=Fun("=", Get.l.id, Get.license_id))
    >> Where(Fun.LIKE(Get.l.car_make, "%Toyota%"))
    >> Select(Get.name, Get.l.age >> As(S.age))
    >> Order(Get.age >> Desc())
)

render_query(query)

query: 
SELECT
  "person_2"."name", 
  "person_2"."age"
FROM (
  SELECT
    "person_1"."name", 
    "drivers_license_1"."age"
  FROM "person" AS "person_1"
  INNER JOIN "drivers_license" AS "drivers_license_1" ON ("drivers_license_1"."id" = "person_1"."license_id")
  WHERE ("drivers_license_1"."car_make" LIKE '%Toyota%')
) AS "person_2"
ORDER BY "person_2"."age" DESC

The Limit node lets us take a slice of the input dataset. Since, the order of output rows is uncertain until you specify an `ORDER BY` clause, Limit is typically used along with an `Order` node. 

_Get the last 10 members who joined the fitness centre._

In [52]:
query = (
    From(S.get_fit_now_member)
    >> Order(Get.membership_start_date >> Desc())
    >> Limit(10)
    >> Select(Get.id, Get.name)
)

render_query(query)

query: 
SELECT
  "get_fit_now_member_1"."id", 
  "get_fit_now_member_1"."name"
FROM "get_fit_now_member" AS "get_fit_now_member_1"
ORDER BY "get_fit_now_member_1"."membership_start_date" DESC
LIMIT 10

## Append and Iterate

The `Append` node concatenates two or more input datasets, similar to the `UNION ALL` clause in SQL. Only the columns that are present in all input datasets is included in the output. 

_Fetch all people who attended an event or visited the fitness centre on the date of crime._

In [53]:
visitors_get_fit = (
    From(S.person)
    >> Join(
        From(S.get_fit_now_member) >> As(S.t1),
        on=Fun("=", Get.t1.person_id, Get.id),
    )
    >> Join(
        From(S.get_fit_now_check_in) >> As(S.t2),
        on=Fun("=", Get.t2.membership_id, Get.t1.id),
    )
    >> Where(Fun("=", Get.t2.check_in_date, Var.DATE))
    >> Select(Get.id, Get.t2.check_in_date)
)

visitors_events = (
    From(S.person)
    >> Join(
        From(S.facebook_event_checkin) >> As(S.t1),
        on=Fun("=", Get.t1.person_id, Get.id),
    )
    >> Where(Fun("=", Get.t1.date, Var.DATE))
    >> Select(Get.id, Get.t1.date)
)

query = visitors_get_fit >> Append(visitors_events) >> Bind(Lit(20180115) >> As(S.DATE))
render_query(query)

query: 
SELECT "person_3"."id"
FROM (
  SELECT
    "person_1"."id", 
    "get_fit_now_check_in_1"."check_in_date"
  FROM "person" AS "person_1"
  INNER JOIN "get_fit_now_member" AS "get_fit_now_member_1" ON ("get_fit_now_member_1"."person_id" = "person_1"."id")
  INNER JOIN "get_fit_now_check_in" AS "get_fit_now_check_in_1" ON ("get_fit_now_check_in_1"."membership_id" = "get_fit_now_member_1"."id")
  WHERE ("get_fit_now_check_in_1"."check_in_date" = 20180115)
) AS "person_3"
UNION ALL
SELECT "person_4"."id"
FROM (
  SELECT
    "person_2"."id", 
    "facebook_event_checkin_1"."date"
  FROM "person" AS "person_2"
  INNER JOIN "facebook_event_checkin" AS "facebook_event_checkin_1" ON ("facebook_event_checkin_1"."person_id" = "person_2"."id")
  WHERE ("facebook_event_checkin_1"."date" = 20180115)
) AS "person_4"

The `Iterate` node lets us specify [recursive CTEs](https://www.sqlite.org/lang_with.html#recursive_common_table_expressions) queries, like those supported by a lot of SQL engines. 

The general form of an iterative query is: `base-query >> Iterate (iterator-query)`, where the `Iterator` query is run repeatedly.

_List the first 20 fibonacci numbers_.

In [54]:
base = Define(Lit(1) >> As(S.n1), Lit(1) >> As(S.n2))
iterator = (
    From(S.fibonacci)
    >> Define(Get.n2 >> As(S.n1), Fun("+", Get.n1, Get.n2) >> As(S.n2))
    >> Limit(20)
    >> As(S.fibonacci)
)

query = base >> Iterate(iterator)
render_query(query)

query: 
WITH RECURSIVE "fibonacci_1" ("n1", "n2")  AS (
  SELECT
    1 AS "n1", 
    1 AS "n2"
  UNION ALL
  SELECT
    "fibonacci_2"."n2" AS "n1", 
    ("fibonacci_2"."n1" + "fibonacci_2"."n2") AS "n2"
  FROM "fibonacci_1" AS "fibonacci_2"
  LIMIT 20
)
SELECT
  "fibonacci_1"."n1", 
  "fibonacci_1"."n2"
FROM "fibonacci_1"

However, the support for `Iterate` is flaky, since the output of FunSQL compiler often runs into circular reference errors. 